## Machine Learning with Keras Neural Networking of categories, text mining, and parts of speech##

Joseph Rochelle

This notebook only includes Part 2 of Exercise 9.4.  I have separate files for Part 1 and Part 3.  I exported the cleaned text dataframe from my Part 1 notebook to a csv to be used in this notebook so that the cleaning code does not need to be re-run multiple times as it is time consuming.


In [1]:
# import libraries
import numpy as np
import pandas as pd
import random


As discussed above, the below code imports the cleaned text dataframe with a parts-of-speech feature and the remainder of this notebook uses this cleaned dataframe.

In [2]:
# read random sample because file is too large 606475 lines 
# https://stackoverflow.com/questions/22258491/read-a-small-random-sample-from-a-big-csv-file-into-a-python-data-frame

p = 0.0016  # .16% of the lines
# if random from [0,1] interval is greater than 0.0016 the row will be skipped
df = pd.read_csv('categComments.csv', 
         skiprows=lambda i: i>0 and random.random() > p
)


In [5]:
# look at what the imported dataframe looks like
df.head()

,cat,txt,pos
0,sports,"['trent', 'brown', 'one', 'piec', 'rais', 'lot...","[[('trent', 'NN'), ('brown', 'IN'), ('one', 'C..."
1,sports,"['he', 'right', 'base', 'franchis', 'tag', 'nu...","[[('he', 'PRP'), ('right', 'VBD'), ('base', 'N..."
2,sports,"['agre', 'fuck']","[[('agre', 'NNS'), ('fuck', 'VBD')]]"
3,sports,"['swap', '1st', 'pat', 'approx', '2000', 'poin...","[[('swap', 'NN'), ('1st', 'CD'), ('pat', 'NN')..."
4,sports,"['I', 'dont', 'know', 'hand', 'Im', 'fulli', '...","[[('I', 'PRP'), ('dont', 'VBP'), ('know', 'JJ'..."


> Will use the part-of-speech feature for the modeling.

In [61]:
# separate into input and output columns (X variable for features and y for target variables)

X = df['pos']
y = df['cat']

In [62]:
# check the feature input
X.head()

0    [[('freeman', 'JJ'), ('pick', 'NN'), ('blitz',...
1    [[('im', 'NN'), ('feel', 'NN'), ('still', 'RB'...
2    [[('bleacher', 'DT'), ('report', 'NN'), ('must...
3                                  [[('delet', 'NN')]]
4    [[('wallpap', 'NN'), ('android', 'NN'), ('pan'...
Name: pos, dtype: object

In [63]:
# convert the target variable to an array
y = np.array(df['cat'])
y[10:15]

array(['science_and_technology', 'science_and_technology',
       'science_and_technology', 'science_and_technology',
       'science_and_technology'], dtype=object)

In [64]:
# Create the tf-idf feature matrix
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features = 3000)
X = tfidf.fit_transform(df['pos'])

In [65]:
# the TfidfVectorizer turned the feature variable to a sparse matrix, which causes problems in the model.
# solve the error by converting the sparse matrix to a dense matrix
X = X.todense()

In [66]:
X.shape

(972, 3000)

In [10]:
# import keras libraries
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

In [47]:
# number of features outputted by the tfidf vectorizer
nFeatures = 3000
# number of categories in the target
nClasses = 3

In [48]:
# build the model 
def build_network():
    """
    Create a function that returns a compiled neural network
    """
    nn = Sequential()
    nn.add(Dense(500, activation = 'relu', input_shape =(nFeatures,)))
    nn.add(Dense(150, activation = 'relu'))
    nn.add(Dense(nClasses, activation = 'softmax'))
    nn.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy']
              )
    return nn

> Train the Keras model and use validation split for splitting the data between training and validation.

In [68]:
# train the model
nn2 = KerasClassifier(build_fn = build_network, 
                            epochs = 200,
                            batch_size = 128)
nn2.fit(X,y, validation_split=0.33)

Epoch 1/200
6/6 [==============================] - 1s 131ms/step - loss: 1.0687 - accuracy: 0.4393 - val_loss: 0.9487 - val_accuracy: 0.9969
Epoch 2/200
6/6 [==============================] - 0s 25ms/step - loss: 0.9389 - accuracy: 0.5745 - val_loss: 0.7401 - val_accuracy: 1.0000
Epoch 3/200
6/6 [==============================] - 0s 22ms/step - loss: 0.7939 - accuracy: 0.5637 - val_loss: 0.5479 - val_accuracy: 1.0000
Epoch 4/200
6/6 [==============================] - 0s 22ms/step - loss: 0.6762 - accuracy: 0.6467 - val_loss: 0.4955 - val_accuracy: 0.9844
Epoch 5/200
6/6 [==============================] - 0s 22ms/step - loss: 0.5485 - accuracy: 0.8848 - val_loss: 0.5119 - val_accuracy: 0.8972
Epoch 6/200
6/6 [==============================] - 0s 27ms/step - loss: 0.4031 - accuracy: 0.9063 - val_loss: 0.4492 - val_accuracy: 0.8785
Epoch 7/200
6/6 [==============================] - 0s 25ms/step - loss: 0.2854 - accuracy: 0.9124 - val_loss: 0.4842 - val_accuracy: 0.7975
Epoch 8/200
6/6 [==

6/6 [==============================] - 0s 25ms/step - loss: 0.0505 - accuracy: 0.9739 - val_loss: 0.5325 - val_accuracy: 0.8037
Epoch 118/200
6/6 [==============================] - 0s 23ms/step - loss: 0.0519 - accuracy: 0.9739 - val_loss: 0.4485 - val_accuracy: 0.8567
Epoch 119/200
6/6 [==============================] - 0s 23ms/step - loss: 0.0536 - accuracy: 0.9724 - val_loss: 0.5397 - val_accuracy: 0.8131
Epoch 120/200
6/6 [==============================] - 0s 23ms/step - loss: 0.0510 - accuracy: 0.9724 - val_loss: 0.7119 - val_accuracy: 0.7259
Epoch 121/200
6/6 [==============================] - 0s 24ms/step - loss: 0.0507 - accuracy: 0.9693 - val_loss: 0.9201 - val_accuracy: 0.6168
Epoch 122/200
6/6 [==============================] - 0s 28ms/step - loss: 0.0521 - accuracy: 0.9693 - val_loss: 0.9007 - val_accuracy: 0.6231
Epoch 123/200
6/6 [==============================] - 0s 28ms/step - loss: 0.0522 - accuracy: 0.9693 - val_loss: 0.8044 - val_accuracy: 0.6480
Epoch 124/200
6/6 [=

Epoch 175/200
6/6 [==============================] - 0s 28ms/step - loss: 0.0507 - accuracy: 0.9739 - val_loss: 0.9559 - val_accuracy: 0.6698
Epoch 176/200
6/6 [==============================] - 0s 25ms/step - loss: 0.0515 - accuracy: 0.9739 - val_loss: 1.0607 - val_accuracy: 0.6417
Epoch 177/200
6/6 [==============================] - 0s 26ms/step - loss: 0.0510 - accuracy: 0.9739 - val_loss: 0.9921 - val_accuracy: 0.6636
Epoch 178/200
6/6 [==============================] - 0s 28ms/step - loss: 0.0506 - accuracy: 0.9739 - val_loss: 0.9308 - val_accuracy: 0.6729
Epoch 179/200
6/6 [==============================] - 0s 30ms/step - loss: 0.0499 - accuracy: 0.9739 - val_loss: 1.0547 - val_accuracy: 0.6480
Epoch 180/200
6/6 [==============================] - 0s 25ms/step - loss: 0.0531 - accuracy: 0.9739 - val_loss: 1.2144 - val_accuracy: 0.6293
Epoch 181/200
6/6 [==============================] - 0s 26ms/step - loss: 0.0549 - accuracy: 0.9739 - val_loss: 1.1895 - val_accuracy: 0.6324
Epoch 

> Make predictions from fitted model.

In [69]:
# make predictions from fitted model
predicts2 = nn2.predict(X)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


> Return measurement calculations.

In [70]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import f1_score 
from sklearn.metrics import cohen_kappa_score 
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import confusion_matrix  

# accuracy: (tp + tn) / (p + n) 
accuracy = accuracy_score(y, predicts2) 
print('Accuracy: %f' % accuracy) 

# precision tp / (tp + fp) 
precision = precision_score(y, predicts2, average = 'macro') 
print('Precision: %f' % precision) 

# recall: tp / (tp + fn) 
recall = recall_score(y, predicts2, average = 'macro') 
print('Recall: %f' % recall) 

# f1: 2 tp / (2 tp + fp + fn) 
f1 = f1_score(y, predicts2, average = 'macro') 
print('F1 score: %f' % f1)   



Accuracy: 0.867284
Precision: 0.876670
Recall: 0.892128
F1 score: 0.868852


>

In [71]:
# confusion matrix 
matrix2 = confusion_matrix(y, predicts2)

In [72]:
# print confusion matrix
print(matrix2)

[[ 30   5   0]
 [  0 251   0]
 [  1 123 562]]
